In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, InvalidSessionIdException, SessionNotCreatedException, NoSuchFrameException
from selenium.webdriver.remote import webelement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import csv
import pandas as pd
import os
import time
import pyperclip as pc
import math
from xml.dom.minidom import Attr
from urllib3.exceptions import MaxRetryError

In [2]:
data_source = "./bto_limits.csv"
url = "https://services2.hdb.gov.sg/webapp/BB29ETHN/BB29STREET"
pd.set_option('display.max_colwidth', None)

# Initialize Scraper

In [3]:
class Scraper:
    def __init__(self, url, data_source):
        chrome_options = Options()
        # chrome_options.add_argument("--headless")
        self.webdriver: webdriver.Safari = webdriver.Safari()
        # self.webdriver: webdriver.Chrome = webdriver.Chrome(options=chrome_options)
        self.webdriver.get(url)
        self.webdriver.implicitly_wait(2)
        self.properties = pd.read_csv(data_source, \
                                                engine='python', \
                                                encoding="ISO-8859-1",
                                                header=0)
        self.enquire_as_map = {'seller': "enqBySeller", 'buyer': "enqByBuyer"}
        self.ethnic_map = {'chinese': '//*[@id="ethGroupChinese"]',
                           'malay': '//*[@id="ethGroupMalay"]',
                           'indian': '//*[@id="ethGroupInd"]'}

    
    def query(self, enquire_as: str, postal_code: int, ethnic_group: str) -> str:
        WebDriverWait(self.webdriver, 5).until(EC.presence_of_element_located((By.ID, 'btnProceed')))
        print(enquire_as, postal_code, ethnic_group)
        self.webdriver.find_element_by_id(self.enquire_as_map[enquire_as]).click()
        self.webdriver.find_element_by_id('postalCde').send_keys(Keys.COMMAND + "a")
        self.webdriver.find_element_by_id('postalCde').send_keys(Keys.DELETE)
        pc.copy(postal_code)
        time.sleep(0.5)
        ActionChains(self.webdriver) \
            .key_down(Keys.COMMAND) \
            .key_down('v') \
            .key_up('v') \
            .key_up(Keys.COMMAND) \
            .perform()
        # print(postal_code)
        # WebDriverWait(self.webdriver, 5).until(EC.presence_of_element_located((By.ID, '//*[@id="ethGroupChinese"]')))
        self.webdriver.find_element_by_xpath(self.ethnic_map[ethnic_group]).click()
        # print(ethnic_group)
        # WebDriverWait(self.webdriver, 5).until(EC.presence_of_element_located((By.ID, '//*[@id="citizenSing"]')))
        self.webdriver.find_element_by_xpath('//*[@id="citizenSing"]').click()
        self.webdriver.find_element_by_id("btnProceed").click()
        time.sleep(2)
        try:
            self.webdriver.find_element_by_xpath('//*[@id="rcdNotFndForm"]/div[2]/div/div/h4')
            print("block doesn't exist")
            self.webdriver.back()
            WebDriverWait(self.webdriver, 5).until(EC.presence_of_element_located((By.ID, "btnProceed")))
            return 'no records found'
        except NoSuchElementException:
            print("has record")
        if "buyer" in enquire_as:
            try:
                WebDriverWait(self.webdriver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="print"]/div[3]/div/table/tbody/tr/td[2]/text()')))
            except NoSuchFrameException:
                print("no such frame lor")
            text = self.webdriver.find_element_by_xpath('//*[@id="print"]/div[3]/div/table/tbody/tr/td[2]/text()').text
        else:
            try:
                WebDriverWait(self.webdriver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="print"]/div[3]/div/div/div/div/p/text()')))
            except NoSuchFrameException:
                print("no such frame lor")
            text = self.webdriver.find_element_by_xpath('//*[@id="print"]/div[3]/div/div/div/div/p/text()').text
        print(text)
        self.webdriver.back()
        WebDriverWait(self.webdriver, 5).until(EC.presence_of_element_located((By.ID, "btnProceed")))
        return text

    def query_HDB(self, index:int, postal_code: int, isNone = False) -> None:
        if isNone:
            for i in self.properties.columns[1:]:
                self.properties.at[index, i] = "Hi"
        else:
            for i in self.properties.columns[1:]:
                print(index)
                ethnic_group, enquire_as = i.split("_")
                res = self.query(enquire_as=enquire_as, postal_code=postal_code, ethnic_group=ethnic_group)
                self.properties.at[index, i] = res

    def query_all(self) -> None:
        for index, row in self.properties.iterrows():
            if row.isnull().any():
                print(index)
            # print(index)
            # print(math.isnan(row.indian_seller))
            # print(row.isnull().any())
            # print(row.indian_seller.isnull())
            if not row.isnull().any():
                continue
            else:
                self.query_HDB(index, row.POSTAL, not row.POSTAL)

In [4]:
# try:
#     scraper.webdriver.quit()
# except NameError:
#     print("crawler not initialised")
# except MaxRetryError:
#     print("crawler quitted / MaxRetryError")
# except InvalidSessionIdException:
#     print("invalid ID")
# except SessionNotCreatedException:
#     print("need to close session")
# except AttributeError:
#     print("manually close")

# scraper = Scraper(url, data_source)
# try:
#     scraper.query_all()
# finally:
#     scraper.properties.to_csv("bto_limits.csv", index=False, encoding="ISO-8859-1")

In [6]:
for i in range(100):
    print(i)
    scraper = Scraper(url, data_source)
    try:
        scraper.query_all()
    except TimeoutException:
        print("timeout :(")
    except NoSuchElementException:
        print("no such element :(")
    except NoSuchFrameException:
        print("no such frame :(")
    finally:
        scraper.properties.to_csv("bto_limits.csv", index=False, encoding="ISO-8859-1")
        scraper.webdriver.quit()

0
535
535
buyer 320119 chinese
has record
You can buy from any flat seller, regardless of their ethnic group and citizenship.
535
seller 320119 chinese
has record
You can only sell your flat to Chinese or Malay buyers. The buyers can be Singapore citizens, Malaysian SPRs or non-Malaysian SPRs.
535
buyer 320119 malay
has record
You can buy from any flat seller, regardless of their ethnic group and citizenship.
535
seller 320119 malay
has record
You can only sell your flat to Malay or Chinese buyers. The buyers can be Singapore citizens, Malaysian SPRs or non-Malaysian SPRs.
535
buyer 320119 indian
has record
You can only buy from Indian and Other Ethnic Groups flat sellers, regardless of their citizenship.
535
seller 320119 indian
has record
You can sell your flat to any buyer, regardless of their ethnic group and citizenship.
536
536
buyer 510119 chinese
has record
You can buy from any flat seller, regardless of their ethnic group and citizenship.
536
seller 510119 chinese
has record
Y

InvalidSessionIdException: Message: 
